In [ ]:
import os
from pandas import read_table
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr, Metadata
from mutperiodpy.GeneratePlotnineFigures import parseAndPlotPeriodicity

jiangDataDirectory = os.path.join(getDataDir(), "Jiang_BPDE_damage_maps")

dataSets = ["BEAS-2B_2uM_BPDE_cell_24h"]

In [ ]:
SINGLE_NUC = 1
NUC_GROUP = 2

jiangRawCounts = dict()
jiangNakedNormalizedCounts = dict()
jiangTrinucNormalizedCounts = dict()

TSS_Names = ["hg19_protein_coding_genes_TSSs"]

for dataSet in dataSets:

    jiangRawCounts[dataSet] = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
    jiangNakedNormalizedCounts[dataSet] = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
    jiangTrinucNormalizedCounts[dataSet] = {SINGLE_NUC:dict(), NUC_GROUP:dict()}

    for nucleosomeCountsFilePath in getFilesInDirectory(jiangDataDirectory, DataTypeStr.rawNucCounts+".tsv"):

        countsMetadata = Metadata(nucleosomeCountsFilePath)
        if countsMetadata.nucPosName not in TSS_Names: continue

        if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
        else: radius = SINGLE_NUC

        jiangRawCounts[dataSet][radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

    for nucleosomeCountsFilePath in getFilesInDirectory(jiangDataDirectory, DataTypeStr.normNucCounts+".tsv"):

        countsMetadata = Metadata(nucleosomeCountsFilePath)
        if countsMetadata.nucPosName not in TSS_Names: continue

        if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
        else: radius = SINGLE_NUC

        if "custom_context" in os.path.basename(nucleosomeCountsFilePath):
            jiangNakedNormalizedCounts[dataSet][radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)
        elif "trinuc" in os.path.basename(nucleosomeCountsFilePath):
            jiangTrinucNormalizedCounts[dataSet][radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

In [ ]:
for dataSet in dataSets:
    for radius in jiangRawCounts[dataSet]:
        for TSS_Name in jiangRawCounts[dataSet][radius]:
            plot = parseAndPlotPeriodicity(jiangRawCounts[dataSet][radius][TSS_Name], dataCol = "Both_Strands_Counts",
                                           smoothTranslational = False, nucRepLen = 2000,
                                           title = f"Jiang {dataSet} {TSS_Name} Raw Counts", yAxisLabel = "Damage Reads")
            print(plot)

In [ ]:
for dataSet in dataSets:
    for radius in jiangNakedNormalizedCounts[dataSet]:
        for TSS_Name in jiangNakedNormalizedCounts[dataSet][radius]:
            plot = parseAndPlotPeriodicity(jiangNakedNormalizedCounts[dataSet][radius][TSS_Name], dataCol = "Normalized_Both_Strands",
                                           smoothTranslational = False, nucRepLen = 2000,
                                           title = f"Jiang {dataSet} {TSS_Name} Naked Normalized", yAxisLabel = "Cellular/Naked Damage Reads")
            print(plot)

In [ ]:
for dataSet in dataSets:
    for radius in jiangTrinucNormalizedCounts[dataSet]:
        for TSS_Name in jiangTrinucNormalizedCounts[dataSet][radius]:
            plot = parseAndPlotPeriodicity(jiangTrinucNormalizedCounts[dataSet][radius][TSS_Name], dataCol = "Normalized_Both_Strands",
                                           smoothTranslational = False, nucRepLen = 2000,
                                           title = f"Jiang {dataSet} {TSS_Name} Trinuc Normalized", yAxisLabel = "Trinuc Normalized Damage")
            print(plot)